---
layout: post
title: Student Teaching Tracker
comments: true
---

<style>
    #student-list {
        list-style: none;
        padding: 0;
        margin: 0;
        color: white !important;
    }

    .student-item {
        display: flex;
        flex-direction: column;
        margin-bottom: 20px;
        color: white !important;
    }

    .student-name {
        text-align: left;
        font-weight: bold;
        color: white !important;
    }

    .controls {
    display: flex;
    justify-content: space-between; /* Space between label and input */
    align-items: center;
    color: white !important;
}

    

    .objectStyled {
        padding: 10px 15px;
        margin: 0;
        font-size: 16px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        border: none;
        outline: none;
        color: white !important;
        background: #1e1e1e;
        border-radius: 10px;
    }

    .inputStyled {
        padding: 10px 15px;
        margin: 0;
        font-size: 16px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        border: none;
        outline: none;
        color: white !important;
        background-color: #444444 !important;
        border-radius: 10px;
    }


    button {
        padding: 10px 15px;
        margin: 0 5px;
        font-size: 16px;
        cursor: pointer;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        color: white !important;
    }

    #control-panel {
        text-align: center;
        margin-top: 20px;
        color: white !important;
    }

    input::file-selector-button {
        font-weight: bold;
        color: dodgerblue;
        padding: 0.5em;
        border: thin solid grey;
        border-radius: 3px;
    }
</style>

<ul id="student-list"></ul>

<div id="control-panel">
    <button class="glow-on-hover-search" onclick="saveSession()">Save Scores</button>
    <button class="glow-on-hover-search" onclick="resetTracker()">Clear Points</button>
    <button type="button" class="glow-on-hover-search" onclick="document.getElementById('file-input-students').click();">Upload Student Roster</button>
    <button type="button" class="glow-on-hover-search" onclick="document.getElementById('file-input-categories').click();">Upload Scoring Categories</button>
    <input type="file" id="file-input-students" onchange="loadFile(event, 'students')" hidden/>
    <input type="file" id="file-input-categories" onchange="loadFile(event, 'categories')" hidden/>
</div>

<script>
    let students = [];
    let categories = [];
    let gradingData = {};

    function loadFile(event, type) {
        const file = event.target.files[0];
        const reader = new FileReader();
        reader.onload = function (e) {
            const content = e.target.result;
            const data = content.split(/\r?\n/).filter(line => line.trim() !== "");

            if (type === 'students') {
                students = data;
                initializeTracker();
            } else if (type === 'categories') {
                categories = data;
                initializeTracker();
            }
        };
        reader.readAsText(file);
    }

    function initializeTracker() {
        if (students.length === 0 || categories.length === 0) return;

        const studentList = document.getElementById('student-list');
        studentList.innerHTML = '';
        students.forEach(student => {
            gradingData[student] = {};
            categories.forEach(category => {
                gradingData[student][category] = 0; // Initialize all categories with 0
            });

            const li = document.createElement('li');
            li.className = 'student-item';
            li.innerHTML = `
                <span class="student-name">${student}</span>
                ${categories.map(category => `
                    <span class="controls">
                        <span>${category}:</span>
                        <input id="input-${student}-${category}" type="number" step="0.01" min="0" max="1" class="inputStyled" value="0">
                    </span>
                `).join('')}
            `;
            li.classList.add("objectStyled")
            studentList.appendChild(li);
        });
    }

    function calculateTotal(student) {
        let total = 0;
        categories.forEach(category => {
            total += parseFloat(document.getElementById(`input-${student}-${category}`).value || 0);
        });
        return total.toFixed(2);
    }

    function saveSession() {
        let fileContent = students.map(student => {
            const studentScores = categories.map(category => {
                const score = parseFloat(document.getElementById(`input-${student}-${category}`).value || 0);
                gradingData[student][category] = score;
                return `${category}: ${score.toFixed(2)}`;
            }).join('\n');
            const totalScore = calculateTotal(student);
            return `${student} total score: ${totalScore}\n${studentScores}`;
        }).join('\n\n');

        const blob = new Blob([fileContent], { type: 'text/plain' });
        const link = document.createElement('a');
        link.href = URL.createObjectURL(blob);
        link.download = 'grading_data.txt';
        link.click();
    }

    function resetTracker() {
        students.forEach(student => {
            categories.forEach(category => {
                document.getElementById(`input-${student}-${category}`).value = 0;
            });
        });
    }
</script>


### Directions
1. Upload a .txt file with a list of student names seperated by newlines
2. Each time a student answers a question, asks one, or does any form of class participation, click the Log Participation button
3. You can view the number of times a student has participated in the Points section
4. When you're finished, click the Save Data button to save this to a text file. In a real scenario, you could give a grade for participation based on the number of times they participated.
5. If you want to reset the number of points everyone has, click the Reset Points button.